In [1]:
%pylab inline
from __future__ import (division, print_function)

import os
import sys
import copy
import fnmatch
import warnings

# Numpy & Scipy
import scipy
from scipy import misc 
import numpy as numpy 
# Astropy related
from astropy.io import fits 
from astropy import wcs
from astropy import units as u
from astropy.table import Table, Column, vstack
from astropy.stats import sigma_clip
from astropy.nddata import Cutout2D
from astropy.utils.console import ProgressBar
from astropy import coordinates as coords

# cPickle for saveing data
import cPickle as pickle 
# Scipy
import scipy.ndimage as ndimage

# Matplotlib 
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
plt.ioff()
from astroML.plotting import hist

# ColorMap
from palettable.colorbrewer.sequential import PuBu_5, OrRd_6
cmap1 = PuBu_5.mpl_colormap
cmap2 = OrRd_6.mpl_colormap
# Cubehelix color scheme from https://github.com/jradavenport/cubehelix
import cubehelix  
cmap3 = cubehelix.cmap(start=0.5, rot=-0.8, gamma=1.0, 
                       minSat=1.2, maxSat=1.2, 
                       minLight=0.0, maxLight=1.0)

# Matplotlib default settings
rcdef = plt.rcParams.copy()
pylab.rcParams['figure.figsize'] = 12, 10
pylab.rcParams['xtick.major.size'] = 8.0
pylab.rcParams['xtick.major.width'] = 1.5
pylab.rcParams['xtick.minor.size'] = 4.0
pylab.rcParams['xtick.minor.width'] = 1.5
pylab.rcParams['ytick.major.size'] = 8.0
pylab.rcParams['ytick.major.width'] = 1.5
pylab.rcParams['ytick.minor.size'] = 4.0
pylab.rcParams['ytick.minor.width'] = 1.5
mpl.rcParams['legend.numpoints'] = 1
rc('axes', linewidth=2)

# Define the region of interests: 
from shapely.geometry import Polygon, Point
from descartes import PolygonPatch

import emcee
import corner

Populating the interactive namespace from numpy and matplotlib


# Select Very Nearby Galaxies in HSC DR15b 

## Using the external spec-z catalog

In [2]:
specz = Table.read('dr15b_specz.fits', format='fits')
print(len(specz))

4037457


In [3]:
print(specz.colnames)

['id', 'id_isnull', 'name', 'name_isnull', 'ra2000', 'ra2000_isnull', 'decl2000', 'decl2000_isnull', 'redshift', 'redshift_isnull', 'redshift_err', 'redshift_err_isnull', 'original_flag', 'original_flag_isnull', 'flag_homogeneous', 'flag_homogeneous_isnull', 'mag_i', 'mag_i_isnull', 'flag_sdss_dr12', 'flag_sdss_dr12_isnull', 'flag_deep2_dr4', 'flag_deep2_dr4_isnull', 'flag_primus_dr1', 'flag_primus_dr1_isnull', 'flag_vipers_pdr1', 'flag_vipers_pdr1_isnull', 'flag_vvds', 'flag_vvds_isnull', 'flag_gama_dr2', 'flag_gama_dr2_isnull', 'flag_wigglez_dr1', 'flag_wigglez_dr1_isnull', 'flag_zcosmos_dr3', 'flag_zcosmos_dr3_isnull', 'flag_udsz_mar2014', 'flag_udsz_mar2014_isnull', 'flag_3dhst_v4_1_5', 'flag_3dhst_v4_1_5_isnull', 'flag_fmos_cosmos_v1_0', 'flag_fmos_cosmos_v1_0_isnull']


In [4]:
near = specz[(specz['redshift'] >= 0.0005) & 
             (specz['redshift'] <= 0.040)]
print(len(near))

117741


In [10]:
objid = []
hscid = []
imag = []
redshift = []

other = []
red2 = []
hsc2 = []
mag2 = []
ra = []
dec = []

for gal in near: 
    name = gal['name']
    if len(name.split(',')) == 1: 
        if 'SDSS' in name: 
            objid.append(int(name.split('-')[2]))
            hscid.append(gal['id'])
            imag.append(gal['mag_i'])
            redshift.append(gal['redshift'])
        else: 
            other.append(name)
            hsc2.append(gal['id'])
            red2.append(gal['redshift'])
            mag2.append(gal['mag_i'])
            ra.append(gal['ra2000'])
            dec.append(gal['decl2000'])
    else:
        sdss = False
        for ii in range(len(name.split(','))):
            temp = name.split(',')[ii]
            if 'SDSS' in temp:
                objid.append(int(temp.split('-')[2]))
                hscid.append(gal['id'])
                imag.append(gal['mag_i'])
                redshift.append(gal['redshift'])
                sdss = True
        if sdss is False: 
            other.append(name)
            hsc2.append(gal['id'])
            red2.append(gal['redshift'])
            mag2.append(gal['mag_i'])
            ra.append(gal['ra2000'])
            dec.append(gal['decl2000']) 
            
print(len(objid))

print(len(other))

114099
3642


In [8]:
sdssNear = Table([hscid, objid, imag, redshift], 
                 names=(['hscid', 'objid', 'hsc_imag', 'redshift']), 
                 meta={'name': 'sdss nearby'})
#sdssNear.write('dr15b_sdss_near.csv', format='csv')
sdssNear.write('dr15b_sdss_near.fits', format='fits', overwrite=True)

In [11]:
otherNear = Table([hsc2, red2, ra, dec, mag2, other], 
                  names=(['hscid', 'redshift', 'ra', 'dec', 'mag_i', 'name']),
                  meta={'name': 'other nearby'})
otherNear.write('dr15b_other_near.fits', format='fits', overwrite=True)

In [73]:
np.linspace(10176, 10188, 13).astype(np.int32)

array([10176, 10177, 10178, 10179, 10180, 10181, 10182, 10183, 10184,
       10185, 10186, 10187, 10188], dtype=int32)